In [ ]:
!nvidia-smi

Sun Nov 20 04:37:50 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2

from keras.models import Model, Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D,ConvLSTM2D
from tensorflow.keras.layers import BatchNormalization
import os
import seaborn as sns

In [ ]:
print(os.listdir("/content/drive/MyDrive/Experiment/Ear/Marge dataset/"))

['Normal', 'AOM']


In [ ]:
SIZE = 128
train_images = []
train_labels = []
for directory_path in glob.glob("/content/drive/MyDrive/Experiment/Ear/Marge dataset/*"):
    label = directory_path.split("\\")[-1]
    print(label)
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        print(img_path)
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        train_images.append(img)
        train_labels.append(label)

train_images = np.array(train_images)
train_labels = np.array(train_labels)

/content/drive/MyDrive/Experiment/Ear/Marge dataset/Normal
/content/drive/MyDrive/Experiment/Ear/Marge dataset/Normal/Normal 187.png
/content/drive/MyDrive/Experiment/Ear/Marge dataset/Normal/Normal 150.png
/content/drive/MyDrive/Experiment/Ear/Marge dataset/Normal/Normal 186.png
/content/drive/MyDrive/Experiment/Ear/Marge dataset/Normal/Normal 196.png
/content/drive/MyDrive/Experiment/Ear/Marge dataset/Normal/Normal 111.png
/content/drive/MyDrive/Experiment/Ear/Marge dataset/Normal/Normal 152.png
/content/drive/MyDrive/Experiment/Ear/Marge dataset/Normal/Normal 151.png
/content/drive/MyDrive/Experiment/Ear/Marge dataset/Normal/Normal 148.png
/content/drive/MyDrive/Experiment/Ear/Marge dataset/Normal/Normal 147.png
/content/drive/MyDrive/Experiment/Ear/Marge dataset/Normal/Normal 149.png
/content/drive/MyDrive/Experiment/Ear/Marge dataset/Normal/Normal 153.png
/content/drive/MyDrive/Experiment/Ear/Marge dataset/Normal/Normal 155.png
/content/drive/MyDrive/Experiment/Ear/Marge dataset/N

In [ ]:
test_images = []
test_labels = []
for directory_path in glob.glob("/content/drive/MyDrive/Experiment/Ear/Marge dataset/*"):
    fruit_label = directory_path.split("\\")[-1]
    for img_path in glob.glob(os.path.join(directory_path, "*.png")):
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.resize(img, (SIZE, SIZE))
        img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
        test_images.append(img)
        test_labels.append(fruit_label)

test_images = np.array(test_images)
test_labels = np.array(test_labels)

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(test_labels)
test_labels_encoded = le.transform(test_labels)
le.fit(train_labels)
train_labels_encoded = le.transform(train_labels)


In [ ]:
x_train, y_train, x_test, y_test = train_images, train_labels_encoded, test_images, test_labels_encoded

In [ ]:
x_train, x_test = x_train / 255.0, x_test / 255.0

In [ ]:
from tensorflow.keras.utils import to_categorical
y_train_one_hot = to_categorical(y_train)
y_test_one_hot = to_categorical(y_test)

In [ ]:
from keras.applications.vgg16 import VGG16
VGG_model = VGG16(weights='imagenet', include_top=False, input_shape=(SIZE, SIZE, 3))

58889256/58889256 [==============================] - 5s 0us/step


In [ ]:
for layer in VGG_model.layers:
	layer.trainable = False

In [ ]:
VGG_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 128, 128, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 128, 128, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 64, 64, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 64, 64, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 64, 64, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 32, 32, 128)       0     

In [ ]:
feature_extractor=VGG_model.predict(x_train)

28/28 [==============================] - 10s 75ms/step


In [ ]:
features = feature_extractor.reshape(feature_extractor.shape[0], -1)

In [ ]:
X_for_RF = features

In [ ]:
print(type(X_for_RF))

<class 'numpy.ndarray'>


In [ ]:

X_for_RF.shape

(876, 8192)

In [ ]:
import numpy as np
import pandas as pd
df = pd.DataFrame(X_for_RF)

In [ ]:
df

0     1     2     3     4     5     6         7     8     9     ...  \
0    0.000000   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   0.0   0.0  ...   
1    0.000000   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   0.0   0.0  ...   
2    0.000000   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   0.0   0.0  ...   
3    0.000000   0.0   0.0   0.0   0.0   0.0   0.0  0.014488   0.0   0.0  ...   
4    0.000000   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   0.0   0.0  ...   
..        ...   ...   ...   ...   ...   ...   ...       ...   ...   ...  ...   
871  0.014065   0.0   0.0   0.0   0.0   0.0   0.0  0.191229   0.0   0.0  ...   
872  0.000000   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   0.0   0.0  ...   
873  0.145710   0.0   0.0   0.0   0.0   0.0   0.0  0.048289   0.0   0.0  ...   
874  0.459817   0.0   0.0   0.0   0.0   0.0   0.0  0.000000   0.0   0.0  ...   
875  0.112805   0.0   0.0   0.0   0.0   0.0   0.0  0.051208   0.0   0.0  ...   

         8182      8183      8184  8185      8186  8187      8188  8189  \
0    1.889004  0.374266  0.067381   0.0  0.000000   0.0  0.098490   0.0   
1    1.888532  0.281259  0.098006   0.0  0.000000   0.0  0.104491   0.0   
2    1.938380  0.310100  0.152014   0.0  0.000000   0.0  0.145964   0.0   
3    1.894754  0.517534  0.158959   0.0  0.091652   0.0  0.013073   0.0   
4    1.874083  0.168346  0.005523   0.0  0.000000   0.0  0.000000   0.0   
..        ...       ...       ...   ...       ...   ...       ...   ...   
871  1.794656  0.197635  0.061046   0.0  0.000000   0.0  0.086967   0.0   
872  2.037233  0.193488  0.000000   0.0  0.000000   0.0  0.085369   0.0   
873  1.938206  0.444089  0.102119   0.0  0.012840   0.0  0.095284   0.0   
874  1.944632  0.188019  0.037453   0.0  0.018218   0.0  0.000000   0.0   
875  1.901899  0.397087  0.000000   0.0  0.000000   0.0  0.051883   0.0   

         8190  8191  
0    0.401987   0.0  
1    0.429600   0.0  
2    0.387088   0.0  
3    0.403170   0.0  
4    0.327473   0.0  
..        ...   ...  
871  0.401664   0.0  
872  0.346420   0.0  
873  0.384854   0.0  
874  0.387976   0.0  
875  0.351059   0.0  

[876 rows x 8192 columns]

In [ ]:
df['class'] = y_train

In [ ]:
df.to_csv('/content/drive/MyDrive/Experiment/Ear/Marge dataset/ear.csv')